Load the data

In [2]:
%pip install pandas
import pandas as pd

Note: you may need to restart the kernel to use updated packages.


In [ ]:
reviewers_df = pd.read_csv('../data/users_with_locations.csv.gz',compression='gzip')

explore the data - not too much class imbalance -- XGboost will take care of this.

In [3]:
# look at class balance of the column location_us
print(reviewers_df['location_us'].value_counts())


location_us
0    333266
1    149620
Name: count, dtype: int64


using spacy lets egineer some more features

In [4]:
%pip install spacy
%pip install re 
%pip install tqdm
%pip install numpy
%pip install -U pip setuptools wheel
%pip install -U spacy
%pip install -U spacy-transformers

Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
import spacy
import re
import numpy as np
from tqdm import tqdm

In [6]:
import subprocess
import sys
subprocess.check_call([sys.executable, "-m", "spacy", "download", "en_core_web_trf"])

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.4/457.4 MB 43.1 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')


0

In [8]:
# Load spaCy transformer model
print("Loading spaCy model...")
nlp = spacy.load("en_core_web_trf")

# US cities and states for validation/lookup
us_states = set([
    "alabama", "alaska", "arizona", "arkansas", "california", "colorado", 
    "connecticut", "delaware", "florida", "georgia", "hawaii", "idaho", 
    "illinois", "indiana", "iowa", "kansas", "kentucky", "louisiana", 
    "maine", "maryland", "massachusetts", "michigan", "minnesota", 
    "mississippi", "missouri", "montana", "nebraska", "nevada", 
    "new hampshire", "new jersey", "new mexico", "new york", "north carolina", 
    "north dakota", "ohio", "oklahoma", "oregon", "pennsylvania", 
    "rhode island", "south carolina", "south dakota", "tennessee", "texas", 
    "utah", "vermont", "virginia", "washington", "west virginia", 
    "wisconsin", "wyoming", "dc", "district of columbia"
])
# Add state abbreviations
state_abbrevs = {
    "al", "ak", "az", "ar", "ca", "co", "ct", "de", "fl", "ga", "hi", "id",
    "il", "in", "ia", "ks", "ky", "la", "me", "md", "ma", "mi", "mn", "ms",
    "mo", "mt", "ne", "nv", "nh", "nj", "nm", "ny", "nc", "nd", "oh", "ok",
    "or", "pa", "ri", "sc", "sd", "tn", "tx", "ut", "vt", "va", "wa", "wv",
    "wi", "wy", "dc"
}
us_states.update(state_abbrevs)


Loading spaCy model...
